In [2]:
# import
import pandas as pd
import pandas as np

In [3]:
# data load
register = pd.read_csv("data/서울시 건축물대장 정보.csv", encoding = "cp949", low_memory = False)
pyojebu = pd.read_csv("data/서울시 건축물대장 표제부 정보.csv", encoding = "cp949", low_memory = False, encoding_errors = "ignore")
chonggual_pyojebu = pd.read_csv("data/서울시 건축물대장 총괄표제부 정보.csv", encoding = "cp949", low_memory = False)
floor_info = pd.read_csv("data/서울특별시_건축물대장_층별개요정보(2019년).csv", encoding = "cp949", low_memory = False)
area_info = pd.read_csv("data/서울시 건축물대장 지역지구구역 정보.csv", encoding = "cp949", low_memory = False)
jeonyubu = pd.read_csv("data/서울시 건축물대장 전유부 정보.csv", encoding = "cp949", low_memory = False)
jeonyubu_area = pd.read_csv("data/서울시 건축물대장 전유공유면적표 정보.csv", encoding = "cp949", low_memory = False)

In [7]:
print(f"--------------------건축물대장 columns name \n {register.columns}")
print(f"--------------------표제부 columns name \n {pyojebu.columns}")
print(f"--------------------총괄표제부 columns name \n {chonggual_pyojebu.columns}")
print(f"--------------------층별개요 columns name \n {floor_info.columns}")
print(f"--------------------지역지구구역 columns name \n {area_info.columns}")
print(f"--------------------전유부 columns name \n {jeonyubu.columns}")
print(f"--------------------전유공유면적표 columns name \n {jeonyubu_area.columns}")

--------------------건축물대장 columns name 
 Index(['관리_건축물대장_PK', '관리_상위_건축물대장_PK', '대장_구분_코드', '대장_종류_코드', '시군구_코드',
       '법정동_코드', '대지_구분_코드', '번', '지', '특수지_명', '블록', '로트', '건물_명',
       '위반_건축물_여부', '대장_일련번호', '총괄표제부_일련번호', '표제부_일련번호', '전유부_일련번호',
       '새주소_도로_코드', '새주소_법정동_코드', '새주소_지상지하_코드', '새주소_본_번', '새주소_부_번',
       '변동_일자'],
      dtype='object')
--------------------표제부 columns name 
 Index(['건축물대장 관리번호', '주용도 구분', '기타 용도', '세대수', '가구수', '구조 구분', '기타 구조',
       '지붕 구분', '기타 지붕', '건축 면적', '연 면적', '용적율 산정 연면적', '높이', '대지 면적', '건폐율',
       '용적율', '지상층수', '지하층수', '승용 승강기수', '비상용 승강기수', '부속 건축물수', '부속 건축물 면적',
       '옥내 자주식 대수', '옥내 자주식 면적', '옥외 자주식 대수', '옥외 자주식 면적', '옥내 기계식 대수',
       '옥내 기계식 면적', '옥외 기계식 대수', '옥외 기계식 면적'],
      dtype='object')
--------------------총괄표제부 columns name 
 Index(['건축물대장 관립번호', '대지 면적', '건축 면적', '건폐율', '연면적', '용적율 산정 연면적', '용적율',
       '주건축물수', '부속 건축물수', '부속 건축물 면적', '세대수', '가구수', '총주차수', '옥내 자주식 대수',
       '옥내 자주식 면적', '옥외 자주식 대수', '옥외 자주식 

In [ ]:
print(f"--------------------건축물대장 shape \n {register.shape}")
print(f"--------------------표제부 shape \n {pyojebu.shape}")
print(f"--------------------총괄표제부 shape \n {chonggual_pyojebu.shape}")
print(f"--------------------층별개요 shape \n {floor_info.shape}")
print(f"--------------------지역지구구역 shape \n {area_info.shape}")
print(f"--------------------전유부 shape \n {jeonyubu.shape}")
print(f"--------------------전유공유면적표 shape \n {jeonyubu_area.shape}")

건축물대장(register) 전처리

In [5]:
# 네 놈이 어떻게 생겼는지 보자
register.head()

,관리_건축물대장_PK,관리_상위_건축물대장_PK,대장_구분_코드,대장_종류_코드,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,...,대장_일련번호,총괄표제부_일련번호,표제부_일련번호,전유부_일련번호,새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번,변동_일자
0,11530-82241,11530-1802,집합,전유부,구로구,구로동,대지,1258,0,NaN,...,2270,1.0,450.0,2270,서울특별시 구로구 경인로53길,구로동,지상,15.0,NaN,20230426
1,11530-118915,11530-15683,집합,전유부,구로구,온수동,대지,10,16,NaN,...,80,1.0,10.0,80,서울특별시 구로구 부일로7길,온수동,지상,13.0,0.0,20230426
2,11530-81288,11530-1802,집합,전유부,구로구,구로동,대지,1258,0,NaN,...,15730,1.0,450.0,15730,서울특별시 구로구 경인로53길,구로동,지상,15.0,NaN,20230426
3,11530-81789,11530-1802,집합,전유부,구로구,구로동,대지,1258,0,NaN,...,19240,1.0,450.0,19240,서울특별시 구로구 경인로53길,구로동,지상,15.0,NaN,20230426
4,11530-80190,11530-1802,집합,전유부,구로구,구로동,대지,1258,0,NaN,...,8490,1.0,450.0,8490,서울특별시 구로구 경인로53길,구로동,지상,15.0,NaN,20230426


In [10]:
# 누가 봐도 "와.. 저건 진짜 필요 없을듯" 하는 column 제거
pre_register = register.drop(["대지_구분_코드", "특수지_명", "블록", "로트", "대장_일련번호", "총괄표제부_일련번호", "표제부_일련번호", "전유부_일련번호", "새주소_법정동_코드", "변동_일자"], axis = 1)
pre_register.head()

,관리_건축물대장_PK,관리_상위_건축물대장_PK,대장_구분_코드,대장_종류_코드,시군구_코드,법정동_코드,번,지,건물_명,위반_건축물_여부,새주소_도로_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번
0,11530-82241,11530-1802,집합,전유부,구로구,구로동,1258,0,중앙유통단지,NaN,서울특별시 구로구 경인로53길,지상,15.0,NaN
1,11530-118915,11530-15683,집합,전유부,구로구,온수동,10,16,NaN,NaN,서울특별시 구로구 부일로7길,지상,13.0,0.0
2,11530-81288,11530-1802,집합,전유부,구로구,구로동,1258,0,중앙유통단지,NaN,서울특별시 구로구 경인로53길,지상,15.0,NaN
3,11530-81789,11530-1802,집합,전유부,구로구,구로동,1258,0,중앙유통단지,NaN,서울특별시 구로구 경인로53길,지상,15.0,NaN
4,11530-80190,11530-1802,집합,전유부,구로구,구로동,1258,0,중앙유통단지,NaN,서울특별시 구로구 경인로53길,지상,15.0,NaN


In [11]:
# 쓸데없이 float인 새주소 int로 변경
pre_register['새주소_본_번'] = pre_register['새주소_본_번'].fillna(0).astype(int)
pre_register['새주소_부_번'] = pre_register['새주소_부_번'].fillna(0).astype(int)

pre_register.head()

,관리_건축물대장_PK,관리_상위_건축물대장_PK,대장_구분_코드,대장_종류_코드,시군구_코드,법정동_코드,번,지,건물_명,위반_건축물_여부,새주소_도로_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번
0,11530-82241,11530-1802,집합,전유부,구로구,구로동,1258,0,중앙유통단지,NaN,서울특별시 구로구 경인로53길,지상,15,0
1,11530-118915,11530-15683,집합,전유부,구로구,온수동,10,16,NaN,NaN,서울특별시 구로구 부일로7길,지상,13,0
2,11530-81288,11530-1802,집합,전유부,구로구,구로동,1258,0,중앙유통단지,NaN,서울특별시 구로구 경인로53길,지상,15,0
3,11530-81789,11530-1802,집합,전유부,구로구,구로동,1258,0,중앙유통단지,NaN,서울특별시 구로구 경인로53길,지상,15,0
4,11530-80190,11530-1802,집합,전유부,구로구,구로동,1258,0,중앙유통단지,NaN,서울특별시 구로구 경인로53길,지상,15,0


In [12]:
# 주소 통합하기 위한 함수 정의
def make_address(row):
    if not pd.isna(row['새주소_도로_코드']):
        if row['새주소_부_번'] == 0:
            return row['새주소_도로_코드'] + ' ' + str(row['새주소_본_번'])
        elif pd.notna(row['새주소_부_번']):
            return row['새주소_도로_코드'] + ' ' + str(row['새주소_본_번']) + '-' + str(row['새주소_부_번'])
    elif pd.notna(row['지']):
        return row['법정동_코드'] + ' ' + str(row['번']) + '-' + str(row['지'])
    else:
        return row['법정동_코드'] + ' ' + str(row['번'])

# apply 함수 사용 -> 전체에 적용
pre_register['주소'] = pre_register.apply(make_address, axis=1)

# 중복을 제거하기 위해 사용한 columns 제거
pre_register.drop(["법정동_코드", "번", "지", "새주소_도로_코드", "새주소_본_번", "새주소_부_번"], axis = 1, inplace = True)
pre_register.head()

# 시군구코드가 과연 필요한가?

,관리_건축물대장_PK,관리_상위_건축물대장_PK,대장_구분_코드,대장_종류_코드,시군구_코드,건물_명,위반_건축물_여부,새주소_지상지하_코드,주소
0,11530-82241,11530-1802,집합,전유부,구로구,중앙유통단지,NaN,지상,서울특별시 구로구 경인로53길 15
1,11530-118915,11530-15683,집합,전유부,구로구,NaN,NaN,지상,서울특별시 구로구 부일로7길 13
2,11530-81288,11530-1802,집합,전유부,구로구,중앙유통단지,NaN,지상,서울특별시 구로구 경인로53길 15
3,11530-81789,11530-1802,집합,전유부,구로구,중앙유통단지,NaN,지상,서울특별시 구로구 경인로53길 15
4,11530-80190,11530-1802,집합,전유부,구로구,중앙유통단지,NaN,지상,서울특별시 구로구 경인로53길 15


In [13]:
# 지하에 해당하는 data 38개 근데 전체는 94만개 ㅋ
pre_register["새주소_지상지하_코드"] = pre_register["새주소_지상지하_코드"].fillna(0).astype(str)
uniq = pre_register["새주소_지상지하_코드"].unique()
base = len(pre_register[pre_register["새주소_지상지하_코드"] == "지하"])
floor = len(pre_register[pre_register["새주소_지상지하_코드"] == "지상"])
nan = len(pre_register[pre_register["새주소_지상지하_코드"] == "0"])

print(f"unique: {uniq} \n 지하 개수: {base} \n 지상 개수: {floor} \n 결측 개수: {nan}")

unique: ['지상' '0' '지하'] 
 지하 개수: 38 
 지상 개수: 948246 
 결측 개수: 113


In [56]:
pre_register.to_csv("pre/건축물대장.csv")

표제부(pyojebu) 전처리

In [14]:
pyojebu.head()

,건축물대장 관리번호,주용도 구분,기타 용도,세대수,가구수,구조 구분,기타 구조,지붕 구분,기타 지붕,건축 면적,...,부속 건축물수,부속 건축물 면적,옥내 자주식 대수,옥내 자주식 면적,옥외 자주식 대수,옥외 자주식 면적,옥내 기계식 대수,옥내 기계식 면적,옥외 기계식 대수,옥외 기계식 면적
0,11110-17349,단독주택,"주택, 정자",0,1,벽돌구조,세멘벽돌조,(철근)콘크리트,경사스라브,0.0,...,1,5.76,0,0.0,0,0.0,0,0.0,0,0.0
1,11140-17469,단독주택,주택,0,1,벽돌구조,연와조,기와,도단즙,0.0,...,0,0.00,0,0.0,0,0.0,0,0.0,0,0.0
2,11140-17747,제2종근린생활시설,근린생활시설,0,0,철근콘크리트구조,"철근콘크리트조, 연와조",(철근)콘크리트,스라브,0.0,...,0,0.00,0,0.0,0,0.0,0,0.0,0,0.0
3,11140-18605,단독주택,주택,0,1,일반목구조,목조,기와,와즙,0.0,...,0,0.00,0,0.0,0,0.0,0,0.0,0,0.0
4,11110-17852,단독주택,주택,0,1,일반목구조,목조,기와,와즙,0.0,...,0,0.00,0,0.0,0,0.0,0,0.0,0,0.0


In [15]:
pre_pyojebu = pyojebu.drop(["구조 구분", "기타 구조", "지붕 구분", "기타 지붕", "건축 면적", "연 면적", "용적율 산정 연면적", 
                            "높이", "대지 면적", "부속 건축물수", "부속 건축물 면적", "옥내 자주식 면적", "옥외 자주식 면적", 
                            "옥내 기계식 면적", "옥외 기계식 면적"], axis = 1)
pre_pyojebu.head()

,건축물대장 관리번호,주용도 구분,기타 용도,세대수,가구수,건폐율,용적율,지상층수,지하층수,승용 승강기수,비상용 승강기수,옥내 자주식 대수,옥외 자주식 대수,옥내 기계식 대수,옥외 기계식 대수
0,11110-17349,단독주택,"주택, 정자",0,1,0.0,0.0,2,1,0,0,0,0,0,0
1,11140-17469,단독주택,주택,0,1,0.0,0.0,1,0,0,0,0,0,0,0
2,11140-17747,제2종근린생활시설,근린생활시설,0,0,0.0,0.0,3,1,0,0,0,0,0,0
3,11140-18605,단독주택,주택,0,1,0.0,0.0,1,0,0,0,0,0,0,0
4,11110-17852,단독주택,주택,0,1,0.0,0.0,1,0,0,0,0,0,0,0


In [16]:
pre_pyojebu["주용도 구분"].unique()

array(['단독주택', '제2종근린생활시설', '공동주택', '숙박시설', '제1종근린생활시설', '교정및군사시설', nan,
       '위락시설', '공장', '교육연구시설', '종교시설', '자동차관련시설', '창고시설', '업무시설',
       '근린생활시설', '문화및집회시설', '노유자시설', '판매시설', '방송통신시설', '교육연구및복지시설',
       '운수시설', '판매및영업시설', '의료시설', '위험물저장및처리시설', '공공용시설', '자원순환관련시설',
       '묘지관련시설', '동.식물관련시설', '운동시설', '수련시설', '관광휴게시설', '분뇨.쓰레기처리시설',
       '장례시설', '발전시설', '야영장시설', '다가구주택'], dtype=object)

In [17]:
tmp = pre_pyojebu[pre_pyojebu["가구수"] != 0]
tmp[tmp["주용도 구분"] == "제1종근린생활시설"]

,건축물대장 관리번호,주용도 구분,기타 용도,세대수,가구수,건폐율,용적율,지상층수,지하층수,승용 승강기수,비상용 승강기수,옥내 자주식 대수,옥외 자주식 대수,옥내 기계식 대수,옥외 기계식 대수
19,11170-11825,제1종근린생활시설,"근린생활시설, 주택",0,1,49.85,180.28,4,1,0,0,0,2,0,0
78,11290-31921,제1종근린생활시설,근린생활시설,0,1,56.50,111.76,2,0,0,0,1,1,0,0
80,11290-31996,제1종근린생활시설,"제1,2종근린생활시설(소매점,의원,학원)",0,1,56.40,219.89,4,1,1,0,0,8,0,0
85,11200-19278,제1종근린생활시설,근린생활시설 주택,0,1,59.02,196.48,4,1,0,0,2,2,0,0
94,11290-9040,제1종근린생활시설,"근린생활시설, 주택",0,1,0.00,218.35,5,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606799,11260-28560,제1종근린생활시설,"근린생활시설,사무실,주택",0,1,0.00,0.00,5,1,0,0,0,0,0,0
606830,11230-29697,제1종근린생활시설,"근린생활시설, 제2종근린생활시설",0,1,0.00,0.00,3,0,0,0,0,0,0,0
606843,11260-7045,제1종근린생활시설,점포및주택,0,1,60.57,144.63,3,1,0,0,0,0,0,0
606847,11230-19226,제1종근린생활시설,"근린생활시설, 단란주점",0,1,59.57,192.95,4,0,0,0,0,2,0,14


In [18]:
# 건축물 대장 조회
pre_register[pre_register["관리_건축물대장_PK"] == "11230-19226"]

,관리_건축물대장_PK,관리_상위_건축물대장_PK,대장_구분_코드,대장_종류_코드,시군구_코드,건물_명,위반_건축물_여부,새주소_지상지하_코드,주소
121250,11230-19226,11230-100216492,일반,일반건축물,동대문구,장안동음식점빌딩,NaN,지상,서울특별시 동대문구 장한로 109


In [19]:
# 건물에 세대나 가구가 있어도 상가가 존재함을 확인 -> 세대 수, 가구 수 column drop
pre_pyojebu.drop(["세대수", "가구수"], axis = 1, inplace = True)
pre_pyojebu

,건축물대장 관리번호,주용도 구분,기타 용도,건폐율,용적율,지상층수,지하층수,승용 승강기수,비상용 승강기수,옥내 자주식 대수,옥외 자주식 대수,옥내 기계식 대수,옥외 기계식 대수
0,11110-17349,단독주택,"주택, 정자",0.0000,0.0000,2,1,0,0,0,0,0,0
1,11140-17469,단독주택,주택,0.0000,0.0000,1,0,0,0,0,0,0,0
2,11140-17747,제2종근린생활시설,근린생활시설,0.0000,0.0000,3,1,0,0,0,0,0,0
3,11140-18605,단독주택,주택,0.0000,0.0000,1,0,0,0,0,0,0,0
4,11110-17852,단독주택,주택,0.0000,0.0000,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
606887,11410-16144,단독주택,다가구용단독주택(6가구),59.1700,177.5100,3,1,0,0,2,0,0,0
606888,11410-24899,제1종근린생활시설,"근린생활시설, 업무시설, 주택",0.0000,0.0000,3,1,0,0,0,0,0,0
606889,11410-2975,단독주택,단독주택(단독주택)및 미술관,24.6000,44.0887,2,1,0,0,0,0,0,0
606890,11410-1000000000000001545085,제2종근린생활시설,제2종근린생활시설(집회장),2.4807,1.8265,2,0,0,0,0,0,0,0


In [20]:
# 사실 기준이 없이 하는 부분인데,,, 승강기 대수, 주차 대수로 승강기 수에 해당하는 2 columns, 주차대수에 해당하는 4 columns 제거
pre_pyojebu["승강기 수"] = pre_pyojebu["비상용 승강기수"] + pre_pyojebu["승용 승강기수"]
pre_pyojebu["주차 대수"] = pre_pyojebu["옥내 기계식 대수"] + pre_pyojebu["옥내 자주식 대수"] + pre_pyojebu["옥외 기계식 대수"] + pre_pyojebu["옥외 자주식 대수"]
pre_pyojebu.drop(["승용 승강기수", "비상용 승강기수", "옥내 자주식 대수", "옥외 자주식 대수", "옥내 기계식 대수", "옥외 기계식 대수"], axis = 1, inplace = True)
pre_pyojebu

,건축물대장 관리번호,주용도 구분,기타 용도,건폐율,용적율,지상층수,지하층수,승강기 수,주차 대수
0,11110-17349,단독주택,"주택, 정자",0.0000,0.0000,2,1,0,0
1,11140-17469,단독주택,주택,0.0000,0.0000,1,0,0,0
2,11140-17747,제2종근린생활시설,근린생활시설,0.0000,0.0000,3,1,0,0
3,11140-18605,단독주택,주택,0.0000,0.0000,1,0,0,0
4,11110-17852,단독주택,주택,0.0000,0.0000,1,0,0,0
...,...,...,...,...,...,...,...,...,...
606887,11410-16144,단독주택,다가구용단독주택(6가구),59.1700,177.5100,3,1,0,2
606888,11410-24899,제1종근린생활시설,"근린생활시설, 업무시설, 주택",0.0000,0.0000,3,1,0,0
606889,11410-2975,단독주택,단독주택(단독주택)및 미술관,24.6000,44.0887,2,1,0,0
606890,11410-1000000000000001545085,제2종근린생활시설,제2종근린생활시설(집회장),2.4807,1.8265,2,0,0,0


In [21]:
# 상가가 들어올 수 있는 조건은 근린생활시설 이어야 한다. 따라서 근린 생활시설 제외 나머지 전부 drop
pre_pyojebu = pre_pyojebu.loc[pre_pyojebu["주용도 구분"].isin(["제2종근린생활시설", "제1종근린생활시설", "근린생활시설"])]

In [22]:
pre_pyojebu

,건축물대장 관리번호,주용도 구분,기타 용도,건폐율,용적율,지상층수,지하층수,승강기 수,주차 대수
2,11140-17747,제2종근린생활시설,근린생활시설,0.0000,0.0000,3,1,0,0
10,11140-20145,제1종근린생활시설,"근린생활시설, 집회소",0.0000,0.0000,4,0,0,0
12,11140-23324,제1종근린생활시설,근린생활시설,0.0000,0.0000,1,0,0,0
15,11140-9246,제2종근린생활시설,"근린생활시설, 주택",59.9800,270.6300,5,1,0,5
18,11110-22799,제2종근린생활시설,"근린생활시설(노래연습장), 근린생활시설(소매점), 사무실, 음식점",0.0000,0.0000,5,1,0,0
...,...,...,...,...,...,...,...,...,...
606879,11590-13065,제1종근린생활시설,영업용,0.0000,0.0000,1,0,0,0
606882,11350-100213357,제1종근린생활시설,제1종근린생활시설,1.3600,1.3100,1,0,0,0
606884,11740-1000000000000001397677,제1종근린생활시설,제1.2종근린생활시설,54.1500,196.7300,5,1,1,4
606888,11410-24899,제1종근린생활시설,"근린생활시설, 업무시설, 주택",0.0000,0.0000,3,1,0,0


In [58]:
pre_pyojebu.to_csv("pre/표제부.csv")

총괄표제부(chonggual_pyojebu) 전처리

In [23]:
chonggual_pyojebu.head()

,건축물대장 관립번호,대지 면적,건축 면적,건폐율,연면적,용적율 산정 연면적,용적율,주건축물수,부속 건축물수,부속 건축물 면적,...,가구수,총주차수,옥내 자주식 대수,옥내 자주식 면적,옥외 자주식 대수,옥외 자주식 면적,옥내 기계식 대수,옥내 기계식 면적,옥외 기계식 대수,옥외 기계식 면적
0,11410-100205741,409.9,244.980,59.770,818.36,818.360,199.650,2,0,0.0,...,0,13,7,187.89,6,0.0,0,0.0,0,0.0
1,11410-100206241,4243.0,1167.543,27.517,7811.37,6947.697,163.745,2,0,0.0,...,0,0,0,0.00,47,0.0,0,0.0,0,0.0
2,11410-100206421,426.0,226.170,53.090,848.70,848.700,199.230,2,0,0.0,...,0,12,9,0.00,3,0.0,0,0.0,0,0.0
3,11215-569,0.0,0.000,0.000,102.66,0.000,0.000,2,0,0.0,...,1,0,0,0.00,0,0.0,0,0.0,0,0.0
4,11215-570,0.0,0.000,0.000,80.55,0.000,0.000,2,0,0.0,...,1,0,0,0.00,0,0.0,0,0.0,0,0.0


In [24]:
pre_chonggual_pyojebu = chonggual_pyojebu.drop(
    ["대지 면적", "건축 면적", "연면적", "용적율 산정 연면적", "부속 건축물수", "부속 건축물 면적","총주차수",  "옥내 자주식 면적", 
     "옥외 자주식 면적", "옥내 기계식 면적", "옥외 기계식 면적"], axis = 1
)
pre_chonggual_pyojebu.head()

,건축물대장 관립번호,건폐율,용적율,주건축물수,세대수,가구수,옥내 자주식 대수,옥외 자주식 대수,옥내 기계식 대수,옥외 기계식 대수
0,11410-100205741,59.770,199.650,2,16,0,7,6,0,0
1,11410-100206241,27.517,163.745,2,0,0,0,47,0,0
2,11410-100206421,53.090,199.230,2,15,0,9,3,0,0
3,11215-569,0.000,0.000,2,0,1,0,0,0,0
4,11215-570,0.000,0.000,2,0,1,0,0,0,0


In [25]:
# 일단 여기도 "주차 대수" column으로 관련된 4 columns 제거
pre_chonggual_pyojebu["주차 대수"] = pre_chonggual_pyojebu["옥내 기계식 대수"] + pre_chonggual_pyojebu["옥내 자주식 대수"] + pre_chonggual_pyojebu["옥외 기계식 대수"] + pre_chonggual_pyojebu["옥외 자주식 대수"]


In [26]:
pre_chonggual_pyojebu.drop(["옥내 자주식 대수", "옥외 자주식 대수", "옥내 기계식 대수", "옥외 기계식 대수"], axis = 1, inplace = True)
pre_chonggual_pyojebu

,건축물대장 관립번호,건폐율,용적율,주건축물수,세대수,가구수,주차 대수
0,11410-100205741,59.770,199.650,2,16,0,13
1,11410-100206241,27.517,163.745,2,0,0,47
2,11410-100206421,53.090,199.230,2,15,0,12
3,11215-569,0.000,0.000,2,0,1,0
4,11215-570,0.000,0.000,2,0,1,0
...,...,...,...,...,...,...,...
19835,11260-3,18.770,302.660,0,716,0,907
19836,11260-308,0.000,0.000,2,0,2,0
19837,11260-313,0.000,0.000,2,0,1,0
19838,11260-32,0.000,0.000,2,0,0,0


In [59]:
pre_chonggual_pyojebu.to_csv("pre/총괄표제부.csv")

In [33]:
# ??? 건축물 대장이랑 관리번호가 안맞는데,,, 싯팔 -> 있는게 있고 없는게 있네
# 건축물 대장 조회
pre_register[pre_register["관리_건축물대장_PK"] == "11200-1000000000000001647527"]

,관리_건축물대장_PK,관리_상위_건축물대장_PK,대장_구분_코드,대장_종류_코드,시군구_코드,건물_명,위반_건축물_여부,새주소_지상지하_코드,주소
61,11200-1000000000000001647527,NaN,일반,총괄표제부,성동구,1,NaN,지상,용답동 108-1


In [31]:
pre_register[pre_register["대장_종류_코드"] == "총괄표제부"]

,관리_건축물대장_PK,관리_상위_건축물대장_PK,대장_구분_코드,대장_종류_코드,시군구_코드,건물_명,위반_건축물_여부,새주소_지상지하_코드,주소
45,11530-602,NaN,집합,총괄표제부,구로구,중앙유통단지,NaN,지상,서울특별시 구로구 경인로53길 15
49,11590-1000000000000000311680,NaN,일반,총괄표제부,동작구,NaN,NaN,0,동작동 123-2
61,11200-1000000000000001647527,NaN,일반,총괄표제부,성동구,1,NaN,지상,용답동 108-1
875,11530-1000000000000000286634,NaN,일반,총괄표제부,구로구,고려대학교부속구로병원,NaN,0,서울특별시 구로구 구로동로 148
876,11530-1000000000000000132714,NaN,일반,총괄표제부,구로구,NaN,NaN,0,서울특별시 구로구 구로동로17길 62
...,...,...,...,...,...,...,...,...,...
946470,11560-48,NaN,집합,총괄표제부,영등포구,삼호한숲아파트,NaN,지상,서울특별시 영등포구 선유로47길 34
947418,11200-600,NaN,일반,총괄표제부,성동구,NaN,위반건축물,지상,성수동2가 223-9
947789,11380-100196693,NaN,일반,총괄표제부,은평구,학교법인숭실학원,NaN,지상,서울특별시 은평구 은평터널로7길 6
948256,11305-419,NaN,일반,총괄표제부,강북구,본당,NaN,지상,서울특별시 강북구 삼양로97길 1


건축물대장 층별개요정보(floor_info) 전처리

In [83]:
floor_info.head()

,관리_층별_개요_PK,관리_건축물대장_PK,관리_주_건축물대장_PK,주_부속_구분_코드,주_부속_일련번호,층_구분_코드,층_번호,층_번호_명,구조_코드,기타_구조,주_용도_코드,기타_용도,면적
0,11110-39088,11110-12767,NaN,0.0,1,20.0,1,1층,11.0,세멘연와조,01001,주택,66.12
1,11110-100140524,11110-100214285,NaN,0.0,1,30.0,1,옥탑1층,21.0,철근콘크리트구조,04402,사무소,46.99
2,11650-18621,11650-4647,NaN,0.0,1,10.0,1,지층,11.0,연와조,01001,NaN,8.56
3,11380-89611,11380-27785,NaN,0.0,1,20.0,3,3층,21.0,철근콘크리트구조,01003,다가구주택,101.84
4,11650-18557,11650-4628,NaN,0.0,1,20.0,2,2층,11.0,연와조,01001,주택,51.90


In [90]:
pre_floor_info = floor_info.drop(["주_부속_일련번호", "구조_코드", "기타_구조"], axis = 1)
# 주_용도_코드: 공통 코드 CM024
pre_floor_info

,관리_층별_개요_PK,관리_건축물대장_PK,관리_주_건축물대장_PK,주_부속_구분_코드,층_구분_코드,층_번호,층_번호_명,주_용도_코드,기타_용도,면적
0,11110-39088,11110-12767,NaN,0.0,20.0,1,1층,01001,주택,66.12
1,11110-100140524,11110-100214285,NaN,0.0,30.0,1,옥탑1층,04402,사무소,46.99
2,11650-18621,11650-4647,NaN,0.0,10.0,1,지층,01001,NaN,8.56
3,11380-89611,11380-27785,NaN,0.0,20.0,3,3층,01003,다가구주택,101.84
4,11650-18557,11650-4628,NaN,0.0,20.0,2,2층,01001,주택,51.90
...,...,...,...,...,...,...,...,...,...,...
845035,11380-117791,11380-34727,NaN,0.0,20.0,2,2층,01001,주택,32.46
845036,11560-62358,11560-17826,NaN,0.0,20.0,1,1층,17100,공장,306.12
845037,11380-117807,11380-34732,NaN,0.0,20.0,1,1층,01001,주택,80.10
845038,11170-100149541,11170-100217500,NaN,0.0,20.0,1,1층,04402,사무소,80.69


In [92]:
pre_floor_info["주_용도_코드"] = pre_floor_info["주_용도_코드"].fillna(0).astype(str)

In [93]:
def format_code(code):
    if str(code)[0] == 'Z':
        num = int(code[1:]) // 100
        return f"Z{num:04d}"
    else:
        return f"{int(code)//1000*1000:05d}"

pre_floor_info["주_용도_코드"] = pre_floor_info["주_용도_코드"].apply(format_code)
pre_floor_info

,관리_층별_개요_PK,관리_건축물대장_PK,관리_주_건축물대장_PK,주_부속_구분_코드,층_구분_코드,층_번호,층_번호_명,주_용도_코드,기타_용도,면적
0,11110-39088,11110-12767,NaN,0.0,20.0,1,1층,01000,주택,66.12
1,11110-100140524,11110-100214285,NaN,0.0,30.0,1,옥탑1층,04000,사무소,46.99
2,11650-18621,11650-4647,NaN,0.0,10.0,1,지층,01000,NaN,8.56
3,11380-89611,11380-27785,NaN,0.0,20.0,3,3층,01000,다가구주택,101.84
4,11650-18557,11650-4628,NaN,0.0,20.0,2,2층,01000,주택,51.90
...,...,...,...,...,...,...,...,...,...,...
845035,11380-117791,11380-34727,NaN,0.0,20.0,2,2층,01000,주택,32.46
845036,11560-62358,11560-17826,NaN,0.0,20.0,1,1층,17000,공장,306.12
845037,11380-117807,11380-34732,NaN,0.0,20.0,1,1층,01000,주택,80.10
845038,11170-100149541,11170-100217500,NaN,0.0,20.0,1,1층,04000,사무소,80.69


In [116]:
pre_floor_info = pre_floor_info[pre_floor_info["주_용도_코드"].isin(["03000", "04000", "Z3000"])]
pre_floor_info.replace({"03000" : "제1종근린생활시설", "04000" : "제2종근린생활시설", "Z3000" : "근린생활시설"}, inplace = True)
pre_floor_info

/var/folders/3f/97dc0f4132b4wnjnn7ms7g3c0000gn/T/ipykernel_3493/2195419608.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pre_floor_info.replace({"03000" : "제1종근린생활시설", "04000" : "제2종근린생활시설", "Z3000" : "근린생활시설"}, inplace = True)


,관리_층별_개요_PK,관리_건축물대장_PK,관리_주_건축물대장_PK,주_부속_구분_코드,층_구분_코드,층_번호,층_번호_명,주_용도_코드,기타_용도,면적
1,11110-100140524,11110-100214285,NaN,0.0,30.0,1,옥탑1층,제2종근린생활시설,사무소,46.99
12,11620-100177723,11620-100258968,NaN,0.0,20.0,3,3층,제2종근린생활시설,다중생활시설,180.06
16,11710-120164,11710-20420,NaN,0.0,20.0,4,4층,제2종근린생활시설,사무실,0.00
18,11620-100177724,11620-100258968,NaN,0.0,20.0,1,1층,제2종근린생활시설,기타사무소,132.29
30,11260-100149596,11260-100245130,NaN,0.0,20.0,3,3층,제2종근린생활시설,근린생활시설-기타사무소,55.71
...,...,...,...,...,...,...,...,...,...,...
845023,11320-61819,11320-14685,NaN,0.0,20.0,1,1층,제1종근린생활시설,소매점,81.00
845024,11215-80681,11215-19941,NaN,0.0,20.0,1,1층,제1종근린생활시설,주차장,36.48
845029,11215-2143,11215-1256,NaN,0.0,20.0,2,2층,제2종근린생활시설,근린생활시설(사무소),102.93
845034,11650-100192779,11650-100278895,NaN,0.0,20.0,1,1층,제2종근린생활시설,공용공간 및 주차장,88.74


In [109]:
pre_floor_info[pre_floor_info["주_부속_구분_코드"] == 1.0]
#pre_floor_info["주_부속_구분_코드"].unique()

,관리_층별_개요_PK,관리_건축물대장_PK,관리_주_건축물대장_PK,주_부속_구분_코드,층_구분_코드,층_번호,층_번호_명,주_용도_코드,기타_용도,면적
224,11170-23605,11170-5948,NaN,1.0,10.0,1,지1,20000,차고,33.5900
352,11260-76359,11260-20617,NaN,1.0,20.0,1,1층,01000,변소,3.3000
400,11230-114060,11230-35847,NaN,1.0,20.0,1,1층,04000,변소,1.1900
424,11230-90055,11230-28479,NaN,1.0,20.0,1,1층,01000,주택,1.9200
436,11230-50339,11230-16143,NaN,1.0,20.0,1,1층,01000,물치,1.9800
...,...,...,...,...,...,...,...,...,...,...
844606,11170-100149280,11170-100217327,11170-100217325,1.0,10.0,2,지2층,02000,경로당,155.9350
844608,11170-100149290,11170-100217330,11170-100217325,1.0,20.0,1,1층,02000,경비실,22.9380
844612,11170-100149291,11170-100217330,11170-100217325,1.0,10.0,1,지1층,02000,"경비실,문주",132.0511
844711,11170-100149299,11170-100217332,11170-100217325,1.0,10.0,2,지2층,04000,"근린생활시설,판매시설,업무시설 기계실",274.8972


In [110]:
pre_floor_info[pre_floor_info["관리_건축물대장_PK"] == "11110-100214285"]

,관리_층별_개요_PK,관리_건축물대장_PK,관리_주_건축물대장_PK,주_부속_구분_코드,층_구분_코드,층_번호,층_번호_명,주_용도_코드,기타_용도,면적
1,11110-100140524,11110-100214285,NaN,0.0,30.0,1,옥탑1층,04000,사무소,46.99
28131,11110-100140527,11110-100214285,NaN,0.0,20.0,3,3층,04000,사무소,46.99
28132,11110-100140526,11110-100214285,NaN,0.0,20.0,2,2층,04000,사무소,46.99
28133,11110-100140525,11110-100214285,NaN,0.0,20.0,1,1층,03000,소매점,36.10


In [111]:
# 건축물 대장 조회
pre_register[pre_register["관리_건축물대장_PK"] == "11170-100217500"]

,관리_건축물대장_PK,관리_상위_건축물대장_PK,대장_구분_코드,대장_종류_코드,시군구_코드,건물_명,위반_건축물_여부,새주소_지상지하_코드,주소


In [40]:
# 아니 왜 층별 자료는 있는데 건축물 대장이 없지

In [118]:
pre_floor_info.to_csv("pre/층별개요.csv")

건축물대장 지역지구구역(area_info) 전처리

In [119]:
area_info.head()

,관리_지역지구구역_pk,관리_건축물대장_pk,지역지구구역_구분_코드,지역지구구역_코드,대표_여부,기타_지역지구구역
0,11110-1,11110-957,용도지역코드,NaN,대표,일반주거지역
1,11110-10,11110-1007,용도지구코드,NaN,대표,최고고도지구
2,11110-100,11110-1206,용도지역코드,일반상업지역,대표,일반상업지역
3,11110-1000,11110-3432,용도지구코드,NaN,NaN,고도최고지구
4,11110-10000,11110-19856,용도지구코드,NaN,대표,주차장정비지구


In [120]:
pre_area_info = area_info.drop(["관리_지역지구구역_pk"], axis = 1)
pre_area_info.head()

,관리_건축물대장_pk,지역지구구역_구분_코드,지역지구구역_코드,대표_여부,기타_지역지구구역
0,11110-957,용도지역코드,NaN,대표,일반주거지역
1,11110-1007,용도지구코드,NaN,대표,최고고도지구
2,11110-1206,용도지역코드,일반상업지역,대표,일반상업지역
3,11110-3432,용도지구코드,NaN,NaN,고도최고지구
4,11110-19856,용도지구코드,NaN,대표,주차장정비지구


In [135]:
pre_area_info[pre_area_info["기타_지역지구구역"].isna() == True]

,관리_건축물대장_pk,지역지구구역_구분_코드,지역지구구역_코드,대표_여부,기타_지역지구구역
5,11110-1000000000000000060121,용도구역코드,NaN,NaN,NaN
6,11110-1000000000000000060121,용도구역코드,NaN,대표,NaN
7,11110-1000000000000000060121,용도지구코드,NaN,대표,NaN
8,11110-1000000000000000060121,용도지역코드,일반상업지역,NaN,NaN
9,11110-1000000000000000060121,용도지역코드,NaN,NaN,NaN
...,...,...,...,...,...
774678,11740-10848,용도지구코드,NaN,대표,NaN
774761,11740-10926,용도구역코드,NaN,대표,NaN
774794,11740-10962,용도지역코드,제3종일반주거지역,대표,NaN
774988,11740-3463,용도구역코드,NaN,대표,NaN


In [128]:
pre_area_info[pre_area_info["지역지구구역_코드"] == "UOH100"]

,관리_건축물대장_pk,지역지구구역_구분_코드,지역지구구역_코드,대표_여부,기타_지역지구구역
3720,11110-100200457,2,UOH100,1,문화지구


In [43]:
pre_area_info[pre_area_info["대표_여부"] == "0"]

,관리_건축물대장_pk,지역지구구역_구분_코드,지역지구구역_코드,대표_여부,기타_지역지구구역
3766,11110-100200797,1,UQA122,0,제2종일반주거지역
4227,11110-100203858,1,UQA111,0,제1종전용주거지역
4426,11110-100204639,3,UOA120,0,상대정화구역
9305,11110-22075,1,110,0,도시지역
9755,11110-22888,2,NaN,0,최고고도지구
...,...,...,...,...,...
768137,11740-467,1,1030,0,준주거지역
768739,11740-5061,1,1020,0,일반주거지역
771252,11740-2451,2,101,0,2종미관지구
771283,11740-2454,2,101,0,2종미관지구


지역지구구역별 용적률의 최고치가 존재하는데 그를 바탕으로 초과 용적률과 미활용 용적률이라는 변수를 생성할 수 있다. </br>
근데 용적률이라는게 높을 수록 좋은거라 두 변수 모두 중소형 빌딩의 매매시에 통계적으로 유의미한 영향을 준다는 논문이 있다.</br>
신보연(Shin Bo Yeon),and 유선종(Yoo Seon Jong). "용적률 활용이 중소형 빌딩 가격에 미치는 영향에 관한 연구." 주거환경(한국주거환경학회논문집) 16.3 (2018): 195-213.</br>
여기서 208쪽</br>
어떻게 활용할까에 대한 생각은 조금 더 필요

In [44]:
pre_chonggual_pyojebu[pre_chonggual_pyojebu["건축물대장 관립번호"] == "11110-19856"]

,건축물대장 관립번호,건폐율,용적율,주건축물수,세대수,가구수,주차 대수


전유부(jeonyubu) 전처리

In [45]:
jeonyubu.head() 

,관리_건축물대장_PK,동명칭,호_명,층_구분_코드
0,11110-73094,NaN,B01호,지하
1,11110-60758,사직동다세대주택,301호,지상
2,11110-60761,NaN,401호,지상
3,11530-29716,107동,503호,지상
4,11530-29717,107동,504호,지상


In [46]:
# 할게 없지만 변수명 통일
pre_jeonyubu = jeonyubu

In [47]:
pre_jeonyubu[pre_jeonyubu["관리_건축물대장_PK"] == "11680-100262797"]

,관리_건축물대장_PK,동명칭,호_명,층_구분_코드
1558692,11680-100262797,NaN,107,지상


In [48]:
# 건축물 대장 조회
pre_register[pre_register["대장_종류_코드"] == "전유부"]

,관리_건축물대장_PK,관리_상위_건축물대장_PK,대장_구분_코드,대장_종류_코드,시군구_코드,건물_명,위반_건축물_여부,새주소_지상지하_코드,주소
0,11530-82241,11530-1802,집합,전유부,구로구,중앙유통단지,NaN,지상,서울특별시 구로구 경인로53길 15
1,11530-118915,11530-15683,집합,전유부,구로구,NaN,NaN,지상,서울특별시 구로구 부일로7길 13
2,11530-81288,11530-1802,집합,전유부,구로구,중앙유통단지,NaN,지상,서울특별시 구로구 경인로53길 15
3,11530-81789,11530-1802,집합,전유부,구로구,중앙유통단지,NaN,지상,서울특별시 구로구 경인로53길 15
4,11530-80190,11530-1802,집합,전유부,구로구,중앙유통단지,NaN,지상,서울특별시 구로구 경인로53길 15
...,...,...,...,...,...,...,...,...,...
948389,11710-100558917,11710-100558902,집합,전유부,송파구,에스아이팰리스 송파 올림픽공원,NaN,지상,서울특별시 송파구 양재대로 1158
948390,11710-100558921,11710-100558902,집합,전유부,송파구,에스아이팰리스 송파 올림픽공원,NaN,지상,서울특별시 송파구 양재대로 1158
948391,11710-100558918,11710-100558902,집합,전유부,송파구,에스아이팰리스 송파 올림픽공원,NaN,지상,서울특별시 송파구 양재대로 1158
948392,11710-100558922,11710-100558902,집합,전유부,송파구,에스아이팰리스 송파 올림픽공원,NaN,지상,서울특별시 송파구 양재대로 1158


In [136]:
pre_jeonyubu.to_csv("pre/전유부.csv")

전유공유면적표(jenyubu_area) 전처리

In [157]:
jeonyubu_area.head()

,관리_전유_공용_면적_pk,호별명세_pk,평형_구분_명,전유_공용_구분_코드,주_부속_구분_코드,층_구분_코드,층_번호,구조_코드,주_용도_코드,기타_용도,면적,작업_일자
0,11000-2778,11000-26,210,2.0,0.0,NaN,0,42.0,Z6999,"복도,화장실",26.320,20230427
1,11215-100104314,11215-100100740,B27.91,2.0,0.0,40.0,0,21.0,02003,도시형생활주택(단지형다세대주택(계단실)),4.950,20220722
2,11230-100083014,11230-100094031,502호(103동),2.0,0.0,40.0,0,21.0,02003,계단실,9.296,20220304
3,11000-2779,11000-26,210,2.0,0.0,40.0,0,42.0,Z6999,"계단실,홀,ELEV.",8.850,20230427
4,11260-100084701,11260-100041098,A-501,1.0,0.0,20.0,5,21.0,02003,1세대,47.080,20210414


In [158]:
pre_jeonyubu_area = jeonyubu_area.drop(["평형_구분_명", "구조_코드", "작업_일자"], axis = 1)
pre_jeonyubu_area.head()

,관리_전유_공용_면적_pk,호별명세_pk,전유_공용_구분_코드,주_부속_구분_코드,층_구분_코드,층_번호,주_용도_코드,기타_용도,면적
0,11000-2778,11000-26,2.0,0.0,NaN,0,Z6999,"복도,화장실",26.320
1,11215-100104314,11215-100100740,2.0,0.0,40.0,0,02003,도시형생활주택(단지형다세대주택(계단실)),4.950
2,11230-100083014,11230-100094031,2.0,0.0,40.0,0,02003,계단실,9.296
3,11000-2779,11000-26,2.0,0.0,40.0,0,Z6999,"계단실,홀,ELEV.",8.850
4,11260-100084701,11260-100041098,1.0,0.0,20.0,5,02003,1세대,47.080


In [159]:
pre_jeonyubu_area["주_용도_코드"].unique()

array(['Z6999', '02003', '14202', '04999', '04001', '04402', '17999',
       '03005', '14299', '03027', '14204', '07201', '02001', '03029',
       '03002', '04002', '03203', '03001', '10003', '02005', '04403',
       '01003', '04105', '02002', '02006', '03202', '04010', '01001', nan,
       '20001', '04102', '04406', '03999', '03199', '04005', '03013',
       '09104', '03021', '03015', '05299', '04499', '03014', '15002',
       '15206', '04100', '10201', '07999', '04107', '18999', '04020',
       '04040', '07102', '04043', '03003', '04036', '13011', '04014',
       '05199', '13999', '13006', '04021', '15101', '14201', '03025',
       '04047', '07212', '04016', '04103', '04003', '02007', '04304',
       '04012', '01002', '05999', '10999', '15999', '09107', '04009',
       '04008', '11201', '05499', '05107', '07199', '04201', '03024',
       '11199', '04299', '11103', '04018', '07001', '03020', '07399',
       '06101', '09103', '04006', '09999', '02004', '03113', '24101',
       '13104',

In [160]:
pre_jeonyubu_area["주_용도_코드"] = pre_jeonyubu_area["주_용도_코드"].fillna(0).astype(str)
pre_jeonyubu_area["주_용도_코드"] = pre_jeonyubu_area["주_용도_코드"].apply(format_code)
pre_jeonyubu_area

,관리_전유_공용_면적_pk,호별명세_pk,전유_공용_구분_코드,주_부속_구분_코드,층_구분_코드,층_번호,주_용도_코드,기타_용도,면적
0,11000-2778,11000-26,2.0,0.0,NaN,0,Z0069,"복도,화장실",26.320
1,11215-100104314,11215-100100740,2.0,0.0,40.0,0,02000,도시형생활주택(단지형다세대주택(계단실)),4.950
2,11230-100083014,11230-100094031,2.0,0.0,40.0,0,02000,계단실,9.296
3,11000-2779,11000-26,2.0,0.0,40.0,0,Z0069,"계단실,홀,ELEV.",8.850
4,11260-100084701,11260-100041098,1.0,0.0,20.0,5,02000,1세대,47.080
...,...,...,...,...,...,...,...,...,...
521024,11000-3844,11000-31,2.0,0.0,10.0,4,02000,"기계실,전기실,발전기실",1.847
521025,11000-3845,11000-31,2.0,0.0,10.0,4,02000,주차장,9.771
521026,11000-3846,11000-31,2.0,0.0,10.0,3,02000,주차장,14.507
521027,11000-3847,11000-31,2.0,0.0,10.0,2,02000,주차장,14.507


In [161]:
pre_jeonyubu_area = pre_jeonyubu_area[pre_jeonyubu_area["주_용도_코드"].isin(["03000", "04000", "Z3000"])]
pre_jeonyubu_area.replace({"03000" : "제1종근린생활시설", "04000" : "제2종근린생활시설", "Z3000" : "근린생활시설"}, inplace = True)
pre_jeonyubu_area

/var/folders/3f/97dc0f4132b4wnjnn7ms7g3c0000gn/T/ipykernel_3493/4109981036.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pre_jeonyubu_area.replace({"03000" : "제1종근린생활시설", "04000" : "제2종근린생활시설", "Z3000" : "근린생활시설"}, inplace = True)


,관리_전유_공용_면적_pk,호별명세_pk,전유_공용_구분_코드,주_부속_구분_코드,층_구분_코드,층_번호,주_용도_코드,기타_용도,면적
23,11260-100088962,11260-100032254,2.0,0.0,40.0,0,제2종근린생활시설,주차장,20.710
33,11230-100073338,11230-100081430,1.0,0.0,20.0,1,제2종근린생활시설,NaN,36.955
40,11230-100073339,11230-100081430,1.0,0.0,20.0,1,제2종근린생활시설,NaN,36.955
42,11530-100096902,11530-100090607,2.0,0.0,20.0,0,제2종근린생활시설,엘레베이터,1.407
45,11260-100088963,11260-100032254,2.0,0.0,40.0,0,제2종근린생활시설,"전기실,발전기실,펌프실(지3),관리실,방재실,MDF(1층),복도,계단실,승강기홀,화장실",20.130
...,...,...,...,...,...,...,...,...,...
520945,11620-100119980,11620-100074232,2.0,0.0,40.0,0,제1종근린생활시설,"코어1,2, EPS/TPS, 복도",43.400
520946,11620-100119981,11620-100074232,2.0,0.0,40.0,0,제1종근린생활시설,"코어1,2, EPS/TPS, 복도",21.700
521017,11000-3831,11000-31,2.0,0.0,NaN,0,제1종근린생활시설,"기계,전기,발전기실",58.290
521018,11000-3837,11000-31,2.0,0.0,NaN,0,제1종근린생활시설,"기타공유(계단실,ELEV.코아)",3.700


In [162]:
len(pre_jeonyubu_area[pre_jeonyubu_area["전유_공용_구분_코드"] == 1.0])

21937

In [52]:
jeonyubu[jeonyubu["관리_건축물대장_PK"] == "11000-31"]

,관리_건축물대장_PK,동명칭,호_명,층_구분_코드


In [ ]:
pre_jeonyubu_area.to_csv("pre/전유공용면적.csv")

In [53]:
print(f"--------------------전처리 건축물대장 columns name \n {pre_register.columns}")
print(f"--------------------전처리 표제부 columns name \n {pre_pyojebu.columns}")
print(f"--------------------전처리 총괄표제부 columns name \n {pre_chonggual_pyojebu.columns}")
print(f"--------------------전처리 층별개요 columns name \n {pre_floor_info.columns}")
print(f"--------------------전처리 지역지구구역 columns name \n {pre_area_info.columns}")
print(f"--------------------전처리 전유부 columns name \n {pre_jeonyubu.columns}")
print(f"--------------------전처리 전유공유면적표 columns name \n {pre_jeonyubu_area.columns}")

--------------------전처리 건축물대장 columns name 
 Index(['관리_건축물대장_PK', '관리_상위_건축물대장_PK', '대장_구분_코드', '대장_종류_코드', '시군구_코드',
       '건물_명', '위반_건축물_여부', '새주소_지상지하_코드', '주소'],
      dtype='object')
--------------------전처리 표제부 columns name 
 Index(['건축물대장 관리번호', '주용도 구분', '기타 용도', '건폐율', '용적율', '지상층수', '지하층수', '승강기 수',
       '주차 대수'],
      dtype='object')
--------------------전처리 총괄표제부 columns name 
 Index(['건축물대장 관립번호', '건폐율', '용적율', '주건축물수', '세대수', '가구수', '주차 대수'], dtype='object')
--------------------전처리 층별개요 columns name 
 Index(['관리_층별_개요_PK', '관리_건축물대장_PK', '관리_주_건축물대장_PK', '주_부속_구분_코드', '층_구분_코드',
       '층_번호', '층_번호_명', '주_용도_코드', '기타_용도', '면적'],
      dtype='object')
--------------------전처리 지역지구구역 columns name 
 Index(['관리_건축물대장_pk', '지역지구구역_구분_코드', '지역지구구역_코드', '대표_여부', '기타_지역지구구역'], dtype='object')
--------------------전처리 전유부 columns name 
 Index(['관리_건축물대장_PK', '동명칭', '호_명', '층_구분_코드'], dtype='object')
--------------------전처리 전유공유면적표 columns name 
 Index(['관리_전유_공용_면적_pk', '호별명세_p

In [54]:
print(f"--------------------건축물대장 shape \n {pre_register.shape}")
print(f"--------------------표제부 shape \n {pre_pyojebu.shape}")
print(f"--------------------총괄표제부 shape \n {pre_chonggual_pyojebu.shape}")
print(f"--------------------층별개요 shape \n {pre_floor_info.shape}")
print(f"--------------------지역지구구역 shape \n {pre_area_info.shape}")
print(f"--------------------전유부 shape \n {pre_jeonyubu.shape}")
print(f"--------------------전유공유면적표 shape \n {pre_jeonyubu_area.shape}")

--------------------건축물대장 shape 
 (948397, 9)
--------------------표제부 shape 
 (123535, 9)
--------------------총괄표제부 shape 
 (19840, 7)
--------------------층별개요 shape 
 (845040, 10)
--------------------지역지구구역 shape 
 (775537, 5)
--------------------전유부 shape 
 (3631394, 4)
--------------------전유공유면적표 shape 
 (521029, 9)
